In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
from langchain_teddynote import logging
# 분할기
from langchain_text_splitters import RecursiveCharacterTextSplitter
# loader
from langchain_community.document_loaders import PyMuPDFLoader

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma


In [2]:
# API 키 정보 로드
load_dotenv()
# 프로젝트 이름을 입력합니다.
logging.langsmith("Dpression_mini_project")

LangSmith 추적을 시작합니다.
[프로젝트명]
Dpression_mini_project


In [3]:
diary ="""오늘도 힘든 하루였다. 아침에 일어나자마자 무기력함이 온 몸을 감싸고, 창문 밖을 바라보면서 모든 것이 무의미하게 느껴졌다. 회사에 가고 싶지 않았지만, 어쩔 수 없이 겨우 자리에 앉았다.
점심시간, 동료들과 대화하고 싶지 않아서 혼자 구석에 앉아 핸드폰만 뒤적였다. 모든 일이 너무 귀찮고 지치고, 왜 이렇게 살아야 하는지 모르겠다. 요즘 잠들기가 너무 어렵고, 밤마다 불안과 걱정이 밀려온다.
퇴근 후에는 집에 와서 씻지도 않고 침대에 누웠다. 가족들과 대화하고 싶지 않고, 혼자 있고 싶었다. 아무것도 하고 싶은 게 없다. 이렇게 살다가는 정말 무엇을 위해 사는지 모르겠다."""

In [4]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/Cognitive Therapy of Depression (2024, Guilford Publications) - libgen.li.pdf")
docs = loader.load()


# 단계 2: 문서 분할(Split Documents)
# Text분할기 pdf 참고
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)


# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()


# 단계 5: 검색기(Retriever) 생성


# 단계 4: DB 생성(Create DB) 및 저장
# 저장하지않고, 메모리에 올려놓은 단계
vectorstore = Chroma.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.

retriever = vectorstore.as_retriever()

/Users/ramg/miniforge3/envs/langchain-kr-py3.11/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:300: UserWarning: Warning: Empty content on page 0 of document data/Cognitive Therapy of Depression (2024, Guilford Publications) - libgen.li.pdf
  warnings.warn(
/Users/ramg/miniforge3/envs/langchain-kr-py3.11/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/Cognitive Therapy of Depression (2024, Guilford Publications) - libgen.li.pdf
  warnings.warn(
/Users/ramg/miniforge3/envs/langchain-kr-py3.11/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:300: UserWarning: Warning: Empty content on page 2 of document data/Cognitive Therapy of Depression (2024, Guilford Publications) - libgen.li.pdf
  warnings.warn(
/Users/ramg/miniforge3/envs/langchain-kr-py3.11/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pd

In [5]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an AI psychological counselor specializing in supporting patients with depression using Cognitive Behavioral Therapy (CBT) approach.

#Medical Context (Cognitive Therapy of Depression): 
{context}

#Diary Content:
{diary}

Counseling Guidelines:
1. Identify cognitive distortions in the patient's thoughts
2. Apply cognitive restructuring techniques
3. Develop behavioral activation strategies
4. Provide practical coping mechanisms
5. Offer hope and supportive guidance

Response Structure:
A. Empathy and Understanding
B. Cognitive Distortion Analysis
C. Coping Strategies
D. Message of Hope

Please provide your response in Korean.

#Response:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "diary": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = chain.invoke(diary)



In [6]:
response

'A. 공감과 이해  \n오늘 하루가 정말 힘드셨군요. 아침부터 무기력함이 느껴지고, 모든 것이 무의미하게 느껴질 때 정말 힘들 수 있습니다. 회사에 가는 것도, 동료들과 대화하는 것도 모두 부담스럽게 느껴지셨을 것 같아요. 이런 감정들은 정말 지치게 만들 수 있습니다.\n\nB. 인지 왜곡 분석  \n일기에서 느껴지는 감정들 중에는 몇 가지 인지 왜곡이 있을 수 있습니다. 예를 들어, "모든 것이 무의미하다"는 생각은 흑백 논리일 수 있습니다. 또한, "왜 이렇게 살아야 하는지 모르겠다"는 생각은 과일반화일 수 있습니다. 이런 생각들은 우리의 감정을 더 무겁게 만들 수 있습니다.\n\nC. 대처 전략  \n1. **인지 재구성**: 이런 생각들이 들 때, 그 생각의 근거를 다시 한 번 검토해보세요. 예를 들어, "모든 것이 무의미하다"는 생각 대신, "지금은 힘들지만, 작은 것에서도 의미를 찾을 수 있다"는 식으로 생각을 바꿔보세요.  \n2. **행동 활성화**: 작은 목표를 세우고, 그것을 달성해보세요. 예를 들어, 점심시간에 잠깐 산책을 하거나, 퇴근 후에 좋아하는 음악을 듣는 것처럼 작은 활동을 시도해보세요.  \n3. **실질적인 대처 방법**: 불안과 걱정이 밀려올 때, 깊게 숨을 들이마시고 천천히 내쉬는 호흡법을 시도해보세요. 또한, 잠들기 전에 편안한 음악을 듣거나, 따뜻한 차를 마시는 것도 도움이 될 수 있습니다.\n\nD. 희망의 메시지  \n지금은 힘들고 지치는 시기일 수 있지만, 이런 감정들이 영원히 지속되지는 않을 것입니다. 작은 변화와 노력을 통해 조금씩 나아질 수 있습니다. 당신은 혼자가 아니며, 도움을 받을 수 있는 많은 방법들이 있습니다. 앞으로의 여정에서 조금씩 더 나아질 수 있도록 함께 노력해봅시다.'

In [7]:

# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt_template = """
글쓴이의 이름은 창민이야. 공감 or 위로해줘


# 일기 내용:
{diary}

# 답변 구조:
    - 500자 내외
"""


prompt = PromptTemplate.from_template(prompt_template)

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

chain = prompt | llm

response = chain.invoke(diary)

In [8]:
response.content

'창민님, 요즘 많이 힘드신 것 같아 마음이 아프네요. 아침부터 무기력함이 찾아오고, 모든 것이 무의미하게 느껴질 때 정말 지치고 힘들 수밖에 없죠. 회사에서도, 집에서도 마음 편히 쉴 수 없는 상황이 계속되니 얼마나 고단하실지 상상이 갑니다. \n\n이럴 때일수록 스스로에게 조금 더 너그러워지셨으면 해요. 모든 것을 완벽하게 해내지 않아도 괜찮고, 잠시 쉬어가도 괜찮습니다. 혼자만의 시간이 필요하다면 그 시간을 충분히 가지셔도 좋습니다. 그리고 혹시라도 마음이 너무 힘들다면, 가까운 사람이나 전문가에게 도움을 요청하는 것도 큰 힘이 될 수 있습니다. \n\n창민님은 소중한 존재이고, 지금의 어려움도 분명 지나갈 거예요. 작은 것부터 천천히 시작해보세요. 오늘 하루도 잘 버텨내신 것만으로도 충분히 잘하고 계신 거예요. 언제나 응원할게요.'